# 공통주제
### GSW 가 40년만에 우승한 이유는 무엇인가.
Golden State Warriors는 1975년 이후로 2015년 약 40년 만에 플레이오프 우승을 달성합니다.
2015년 우승 시즌을 기점으로 어떤 이유로 GSW가 우승할 수 있었는지 분석합니다.

이 분석은 '감독' 관점에서 진행됩니다.

## 가설 설정
- 3점슛 성공률이 높으면 승률이 높을 것이다.
- 3점슛 시도 횟수가 많으면 승률이 높을 것이다.

## 분석 과정
1. GSW 시즌 별 승률 계산 및 시각화
2. GSW 시즌 별 3점슛 성공률 계산 및 시각화
3. GSW 시즌 별 3점슛 성공률과 승률의 상관관계 분석
4. GSW 시즌 별 3점슛 시도 횟수 계산 및 시각화
5. GSW 시즌 별 3점슛 시도 횟수와 승률의 상관관계 분석

### 필요 데이터
1. 테이블
    - games
    - games_details
    - teams

2. 컬럼
    - games
        - GAME_ID
        - SEASON
        - HOME_TEAM_ID
        - VISITOR_TEAM_ID
        - HOME_TEAM_WINS
        - PG3_PCT_away
        - PG3_PCT_home
    - games_details
        - GAME_ID
        - TEAM_ID
        - PG3_PCT
        - PG3A

# 데이터 전처리

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# 원본 데이터 로드
games = pd.read_csv("Data/games.csv", parse_dates=["GAME_DATE_EST"])
games_details = pd.read_csv("Data/games_details.csv", low_memory=False)
teams = pd.read_csv("Data/teams.csv")

In [ ]:
# 한글 폰트 설정
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지

In [ ]:
# games 테이블 전처리
games = games.drop_duplicates(subset=['GAME_ID'], keep='first')
games['HOME_TEAM_WINS'] = games['HOME_TEAM_WINS'].dropna()
games['FG3_PCT_home'] = games['FG3_PCT_home'].fillna(0)
games['PTS_home'] = pd.to_numeric(games['PTS_home'], errors='coerce').fillna(0).astype(int)
games['AST_home'] = pd.to_numeric(games['AST_home'], errors='coerce').fillna(0).astype(int)
games['REB_home'] = pd.to_numeric(games['REB_home'], errors='coerce').fillna(0).astype(int)
games['PTS_away'] = pd.to_numeric(games['PTS_away'], errors='coerce').fillna(0).astype(int)
games['AST_away'] = pd.to_numeric(games['AST_away'], errors='coerce').fillna(0).astype(int)
games['REB_away'] = pd.to_numeric(games['REB_away'], errors='coerce').fillna(0).astype(int)
games['GAME_STATUS_TEXT'] = games['GAME_STATUS_TEXT'].astype('string').fillna('')

games.info()

In [ ]:
# games_details 테이블 전처리
games_details['FG3A'] = pd.to_numeric(games_details['FG3A'], errors='coerce').fillna(0).astype(int)
games_details['FG3M'] = pd.to_numeric(games_details['FG3M'], errors='coerce').fillna(0).astype(int)
games_details['FG3_PCT'] = pd.to_numeric(games_details['FG3_PCT'], errors='coerce').fillna(0)
games_details['COMMENT'] = games_details['COMMENT'].astype('string').fillna('')
games_details['MIN'] = games_details['MIN'].astype('string').fillna('00:00')
games_details['TEAM_ABBREVIATION'] = games_details['TEAM_ABBREVIATION'].astype('string').fillna('')
games_details['TEAM_CITY'] = games_details['TEAM_CITY'].astype('string').fillna('')
games_details['PLAYER_NAME'] = games_details['PLAYER_NAME'].astype('string').fillna('')
games_details['NICKNAME'] = games_details['NICKNAME'].astype('string').fillna('')
games_details['START_POSITION'] = games_details['START_POSITION'].astype('string').fillna('')
games_details['OREB'] =  pd.to_numeric(games_details['OREB'], errors='coerce').fillna(0).astype(int)
games_details['DREB'] =  pd.to_numeric(games_details['DREB'], errors='coerce').fillna(0).astype(int)
games_details['AST'] =  pd.to_numeric(games_details['AST'], errors='coerce').fillna(0).astype(int)
games_details['STL'] =  pd.to_numeric(games_details['STL'], errors='coerce').fillna(0).astype(int)
games_details['BLK'] =  pd.to_numeric(games_details['BLK'], errors='coerce').fillna(0).astype(int)
games_details['PF'] =  pd.to_numeric(games_details['PF'], errors='coerce').fillna(0).astype(int)
games_details['PTS'] =  pd.to_numeric(games_details['PTS'], errors='coerce').fillna(0).astype(int)
games_details['FGM'] =  pd.to_numeric(games_details['FGM'], errors='coerce').fillna(0).astype(int)
games_details['FGA'] =  pd.to_numeric(games_details['FGA'], errors='coerce').fillna(0).astype(int)
games_details['FTM'] =  pd.to_numeric(games_details['FTM'], errors='coerce').fillna(0).astype(int)
games_details['FTA'] =  pd.to_numeric(games_details['FTA'], errors='coerce').fillna(0).astype(int)
games_details['TO'] =  pd.to_numeric(games_details['TO'], errors='coerce').fillna(0).astype(int)
games_details['REB'] =  pd.to_numeric(games_details['REB'], errors='coerce').fillna(0).astype(int)
games_details['PLUS_MINUS'] =  pd.to_numeric(games_details['PLUS_MINUS'], errors='coerce').fillna(0).astype(int)

games_details.info()

In [ ]:
# teams 테이블 전처리
teams['ABBREVIATION'] = teams['ABBREVIATION'].astype('string').str.upper()
teams['NICKNAME'] = teams['NICKNAME'].astype('string')
teams['CITY'] = teams['CITY'].astype('string')
teams['ARENA'] = teams['ARENA'].astype('string')
teams['OWNER'] = teams['OWNER'].astype('string')
teams['HEADCOACH'] = teams['HEADCOACH'].astype('string')
teams['DLEAGUEAFFILIATION'] = teams['DLEAGUEAFFILIATION'].astype('string')
teams['GENERALMANAGER'] = teams['GENERALMANAGER'].astype('string')
teams.info()

# 시즌 별 승률 계산
GSW 팀의 ID를 기준으로 games 테이블에서 홈/원정 게임을 구분하여 시즌 별 승률을 계산합니다.

In [ ]:
# gsw team id
gsw_team_id = 1610612744

In [ ]:
# gsw 이긴 홈 경기
home_games = games[(games['HOME_TEAM_ID'] == gsw_team_id)].copy()
home_games['WIN'] = home_games['HOME_TEAM_WINS']
home_games['TEAM_ID'] = gsw_team_id

# gsw 이긴 원정 경기
away_games = games[(games['VISITOR_TEAM_ID'] == gsw_team_id)].copy()
away_games['WIN'] = 1 - away_games['HOME_TEAM_WINS']
away_games['TEAM_ID'] = gsw_team_id

# 홈+원정 경기 합치기
gsw_all = pd.concat([home_games, away_games], ignore_index=True)

In [ ]:
# 시즌별 승률 계산
season_stats = gsw_all.groupby('SEASON').agg({
    'WIN': ['sum', 'count', 'mean']
}).reset_index()

# 컬럼 재정의
season_stats.columns = ['SEASON', 'WINS', 'GAMES', 'WIN_PCT']

# 우승 시즌(2015) 전후 비교
before_2015 = season_stats[season_stats['SEASON'] < 2015]['WIN_PCT'].mean()
season_2015 = season_stats[season_stats['SEASON'] == 2015]['WIN_PCT'].values[0] if 2015 in season_stats['SEASON'].values else None
after_2015 = season_stats[season_stats['SEASON'] > 2015]['WIN_PCT'].mean()

# GSW 시즌 별 승률 그래프
2002년부터 2022년까지 GSW의 시즌 별 승률 그래프 입니다.
2012년부터 승률이 높아지다가 2015년을 기점으로 승률이 최고점을 찍고 2016년을 기점으로 승률이 떨어집니다.
따라서 승률이 높아지는 2012년부터 2016년까지를 범위로 설정합니다.

In [ ]:
# 시즌별 승률 그래프
plt.figure(figsize=(12, 6))
plt.plot(season_stats['SEASON'], season_stats['WIN_PCT'], 
         marker='o', linewidth=2, markersize=8)

# 2015 우승 시즌 기점
if 2015 in season_stats['SEASON'].values:
    plt.axvline(x=2015, color='red', linestyle='--', 
                label='2015 우승 시즌', linewidth=2)

plt.xlabel('시즌', fontsize=12)
plt.ylabel('승률', fontsize=12)
plt.title('Golden State Warriors 시즌별 승률 추이', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

# 1. 3점슛 성공률이 높으면 승률이 높을 것이다.
앞서 설정한 시즌 범위(2012년 ~ 2016년)을 기준으로 3점슛 성공률을 시즌별로 계산합니다.

In [ ]:
# gsw가 홈/원정인지에 따라 컬럼 선택
gsw_all['FG3_PCT'] = gsw_all.apply(
    lambda row: float(row['FG3_PCT_home']) if row['TEAM_ID_home'] == gsw_team_id 
                else float(row['FG3_PCT_away']),
    axis=1
)

gsw_all['WIN'] = gsw_all.apply(
    lambda row: int(row['HOME_TEAM_WINS']) if row['TEAM_ID_home'] == gsw_team_id 
                else int(1 - row['HOME_TEAM_WINS']),
    axis=1
)

# 시즌별 통계
key_seasons = [2012, 2013, 2014, 2015, 2016]
fg3_pct_season_stats = gsw_all[gsw_all['SEASON'].isin(key_seasons)].groupby('SEASON').agg({
    'FG3_PCT': 'mean',   # 평균 3점슛 성공률
    'WIN': 'mean'        # 승률
}).reset_index()

# 1-1. 시즌 별 3점슛 성공률과 승률 상관관계 분석 (시계열)
계산된 시즌 별 승률과 3점슛 성공률을 시계열 그래프로 나타냅니다.
그래프를 살펴보면 2013년부터 2015년까지는 승률과 3점슛 성공률이 연관성을 보이지만, 2012년과 2016년에는 승률과 3점슛 성공률의 연관성이 낮아 보입니다.

In [ ]:
# 그래프 크기 설정
fig, ax1 = plt.subplots(figsize=(14, 7))

# 첫 번째 y축: 승률
color1 = '#1f77b4'
ax1.set_xlabel('시즌', fontsize=13, fontweight='bold')
ax1.set_ylabel('승률', fontsize=13, fontweight='bold', color=color1)
ax1.plot(fg3_pct_season_stats['SEASON'], fg3_pct_season_stats['WIN'], 
         marker='o', linewidth=3, markersize=10, 
         color=color1, label='승률')
ax1.tick_params(axis='y', labelcolor=color1, labelsize=11)
ax1.grid(True, alpha=0.2)

# 두 번째 y축: 3점슛 성공률
ax2 = ax1.twinx()
color2 = '#ff7f0e'
ax2.set_ylabel('3점슛 성공률', fontsize=13, fontweight='bold', color=color2)
ax2.plot(fg3_pct_season_stats['SEASON'], fg3_pct_season_stats['FG3_PCT'], 
         marker='s', linewidth=3, markersize=10, 
         color=color2, label='3점슛 성공률')
ax2.tick_params(axis='y', labelcolor=color2, labelsize=11)

# 2015 우승 시즌 강조
ax1.axvline(x=2015, color='red', linestyle=':', 
            linewidth=3, alpha=0.6, label='2015 우승')

# 제목
plt.title('Golden State Warriors: 3점슛 성공률과 승률의 관계 (2012-2016)', 
          fontsize=15, fontweight='bold', pad=20)

# 범례
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, 
           loc='upper left', fontsize=11, framealpha=0.9)

plt.tight_layout()
plt.show()

# 1-2.시즌 별 3점슛 성공률과 승률 상관관계 분석 (산점도)

- 추세선은 상향하지만 상관관계는 약함: 산점도에서 추세선이 우상향하여 3점슛 성공률이 높을수록 승률도 높아지는 경향을 보이지만, 상관계수가 0.305로 통계적으로 약한 상관관계에 해당합니다.

- 시즌별 불규칙한 분포: 2013~2015년에는 3점슛 성공률과 승률이 함께 상승하는 패턴을 보이지만, 2012년과 2016년은 이 추세에서 벗어나 데이터 포인트가 산발적으로 분포합니다.

결론: GSW의 2012~2016년 데이터만으로는 "3점슛 성공률이 높으면 승률이 높다"는 가설을 강하게 지지하기 어렵습니다. 승률에는 3점슛 성공률 외에 다른 변수들(수비, 리바운드 등)이 복합적으로 작용하는 것으로 보입니다.

In [ ]:
plt.figure(figsize=(10, 8))

# 산점도
plt.scatter(fg3_pct_season_stats['FG3_PCT'], fg3_pct_season_stats['WIN'], 
            s=200, alpha=0.6, c=fg3_pct_season_stats['SEASON'], 
            cmap='viridis', edgecolors='black', linewidth=2)

# 각 점에 시즌 라벨 추가
for idx, row in fg3_pct_season_stats.iterrows():
    plt.annotate(f"{int(row['SEASON'])}", 
                 (row['FG3_PCT'], row['WIN']),
                 fontsize=11, fontweight='bold',
                 ha='center', va='center')

# 추세선 추가
z = np.polyfit(fg3_pct_season_stats['FG3_PCT'], fg3_pct_season_stats['WIN'], 1)
p = np.poly1d(z)
plt.plot(fg3_pct_season_stats['FG3_PCT'], p(fg3_pct_season_stats['FG3_PCT']), 
         "r--", alpha=0.8, linewidth=2, label='추세선')

plt.xlabel('3점슛 성공률', fontsize=13, fontweight='bold')
plt.ylabel('승률', fontsize=13, fontweight='bold')
plt.title('Golden State Warriors: 3점슛 성공률 vs 승률 상관관계', 
          fontsize=15, fontweight='bold')
plt.colorbar(label='시즌')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 상관계수 출력
correlation = fg3_pct_season_stats['FG3_PCT'].corr(fg3_pct_season_stats['WIN'])
print(f"\n📊 상관계수: {correlation:.3f}")
if correlation > 0.7:
    print("→ 강한 양의 상관관계! 3점슛 성공률이 높을수록 승률도 높다! ✅")
elif correlation > 0.4:
    print("→ 중간 정도의 양의 상관관계")
else:
    print("→ 약한 상관관계")

# 2. 3점슛 평균 시도율이 높을 수록 승률이 높을 것이다.
그렇다면, 시즌 별 3점슛 평균 시도율이 승률과 연관성이 있을까?
시즌 별 3점슛 평균 시도율을 계산하고, 이를 승률과 비교 해 보았습니다.

In [ ]:
# gsw 의 games_details
gsw_games_details = games_details[games_details['TEAM_ID'] == gsw_team_id]

# games 데이터와 병합
joined = games.merge(gsw_games_details, on='GAME_ID', how='inner')

# gsw 승패 여부 계산
joined['WIN'] = joined.apply(
    lambda row: int(row['HOME_TEAM_WINS']) if row['TEAM_ID_home'] == gsw_team_id 
                else int(1 - row['HOME_TEAM_WINS']),
    axis=1
)

# 주요 시즌 필터링 (2012-2016)
key_seasons = [2012, 2013, 2014, 2015, 2016]
joined_filtered = joined[joined['SEASON'].isin(key_seasons)]

# 시즌별 3점슛 시도 횟수와 승률 통계
season_fg3a_stats = joined_filtered.groupby('SEASON').agg({
    'FG3A': 'mean',  # 경기당 평균 3점슛 시도
    'WIN': 'mean'    # 승률
}).reset_index()

season_fg3a_stats.columns = ['SEASON', 'FG3A_AVG', 'WIN_PCT']

# 2-1. 시즌 별 3점슛 평균 시도율과 성공률 상관관계 (시계열)

In [ ]:
# 시각화 1: 시즌별 3점슛 시도 횟수와 승률 비교 (이중 Y축)
fig, ax1 = plt.subplots(figsize=(14, 7))

# 첫 번째 y축: 승률
color1 = '#1f77b4'
ax1.set_xlabel('시즌', fontsize=13, fontweight='bold')
ax1.set_ylabel('승률', fontsize=13, fontweight='bold', color=color1)
ax1.plot(season_fg3a_stats['SEASON'], season_fg3a_stats['WIN_PCT'], 
         marker='o', linewidth=3, markersize=10, 
         color=color1, label='승률')
ax1.tick_params(axis='y', labelcolor=color1, labelsize=11)
ax1.grid(True, alpha=0.2)

# 두 번째 y축: 3점슛 시도 횟수
ax2 = ax1.twinx()
color2 = '#2ca02c'
ax2.set_ylabel('경기당 3점슛 시도 횟수', fontsize=13, fontweight='bold', color=color2)
ax2.plot(season_fg3a_stats['SEASON'], season_fg3a_stats['FG3A_AVG'], 
         marker='s', linewidth=3, markersize=10, 
         color=color2, label='3점슛 시도')
ax2.tick_params(axis='y', labelcolor=color2, labelsize=11)

# 2015 우승 시즌 강조
ax1.axvline(x=2015, color='red', linestyle=':', 
            linewidth=3, alpha=0.6, label='2015 우승')

# 제목
plt.title('Golden State Warriors: 3점슛 시도 횟수와 승률의 관계 (2012-2016)', 
          fontsize=15, fontweight='bold', pad=20)

# 범례
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, 
           loc='upper left', fontsize=11, framealpha=0.9)

plt.tight_layout()
plt.show()

# 2-2. 시즌 별 3점슛 평균 시도율과 성공률 상관관계 분석 (산점도)
- 강한 양의 상관관계 확인: 상관계수 0.823으로, 3점슛 시도 횟수와 승률 사이에 강한 양의 상관관계가 존재합니다. 이는 앞서 분석한 3점슛 성공률(0.305)보다 훨씬 높은 수치입니다.

- 시계열 그래프 패턴: 2012년부터 2016년까지 3점슛 시도 횟수와 승률이 거의 동일한 추세로 상승하며, 두 지표가 밀접하게 연동되어 움직이는 것을 확인할 수 있습니다.

- 핵심 인사이트: GSW의 승률 향상에는 3점슛 "성공률"보다 3점슛 "시도 횟수"가 더 큰 영향을 미쳤습니다. 이는 GSW가 적극적인 3점슛 전략을 통해 경기력을 끌어올렸음을 시사합니다.

결론: "많이 쏘는 것"이 "잘 넣는 것"보다 승률에 더 직접적인 영향을 주었으며, 이는 GSW 우승의 핵심 요인이었음을 뒷받침합니다.

In [ ]:
# 시각화 2: 3점슛 시도 횟수 vs 승률 산점도 (상관관계 분석)
plt.figure(figsize=(10, 8))

# 산점도
plt.scatter(season_fg3a_stats['FG3A_AVG'], season_fg3a_stats['WIN_PCT'], 
            s=200, alpha=0.6, c=season_fg3a_stats['SEASON'], 
            cmap='viridis', edgecolors='black', linewidth=2)

# 각 점에 시즌 라벨 추가
for idx, row in season_fg3a_stats.iterrows():
    plt.annotate(f"{int(row['SEASON'])}", 
                 (row['FG3A_AVG'], row['WIN_PCT']),
                 fontsize=11, fontweight='bold',
                 ha='center', va='center')

# 추세선 추가
z = np.polyfit(season_fg3a_stats['FG3A_AVG'], season_fg3a_stats['WIN_PCT'], 1)
p = np.poly1d(z)
x_line = np.linspace(season_fg3a_stats['FG3A_AVG'].min(), season_fg3a_stats['FG3A_AVG'].max(), 100)
plt.plot(x_line, p(x_line), "r--", alpha=0.8, linewidth=2, label='추세선')

plt.xlabel('경기당 3점슛 시도 횟수', fontsize=13, fontweight='bold')
plt.ylabel('승률', fontsize=13, fontweight='bold')
plt.title('Golden State Warriors: 3점슛 시도 횟수 vs 승률 상관관계', 
          fontsize=15, fontweight='bold')
plt.colorbar(label='시즌')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 상관계수 출력
correlation = season_fg3a_stats['FG3A_AVG'].corr(season_fg3a_stats['WIN_PCT'])
print(f"\n📊 3점슛 시도 횟수 vs 승률 상관계수: {correlation:.3f}")
if correlation > 0.7:
    print("→ 강한 양의 상관관계! 3점슛을 많이 시도할수록 승률이 높다!")
elif correlation > 0.4:
    print("→ 중간 정도의 양의 상관관계")
elif correlation > 0:
    print("→ 약한 양의 상관관계")
else:
    print("→ 상관관계가 없거나 음의 상관관계")